# Getting and analysing data from the S&P 500 Index

In [4]:
import bs4 as bs
import datetime as dt
import os
import pandas as pd
import pandas_datareader as web
import pickle
import requests

### Scraping the wikipedia page for the tickers

In [5]:
def save_sp500_tickers():
    resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text, 'lxml')
    table = soup.find('table', {'class': 'wikitable sortable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
        
    return tickers

In [11]:
save_sp500_tickers()

['MMM',
 'ABT',
 'ABBV',
 'ABMD',
 'ACN',
 'ATVI',
 'ADBE',
 'AMD',
 'AAP',
 'AES',
 'AET',
 'AMG',
 'AFL',
 'A',
 'APD',
 'AKAM',
 'ALK',
 'ALB',
 'ARE',
 'ALXN',
 'ALGN',
 'ALLE',
 'AGN',
 'ADS',
 'LNT',
 'ALL',
 'GOOGL',
 'GOOG',
 'MO',
 'AMZN',
 'AEE',
 'AAL',
 'AEP',
 'AXP',
 'AIG',
 'AMT',
 'AWK',
 'AMP',
 'ABC',
 'AME',
 'AMGN',
 'APH',
 'APC',
 'ADI',
 'ANDV',
 'ANSS',
 'ANTM',
 'AON',
 'AOS',
 'APA',
 'AIV',
 'AAPL',
 'AMAT',
 'APTV',
 'ADM',
 'ARNC',
 'ANET',
 'AJG',
 'AIZ',
 'T',
 'ADSK',
 'ADP',
 'AZO',
 'AVB',
 'AVY',
 'BHGE',
 'BLL',
 'BAC',
 'BK',
 'BAX',
 'BBT',
 'BDX',
 'BRK.B',
 'BBY',
 'BIIB',
 'BLK',
 'HRB',
 'BA',
 'BKNG',
 'BWA',
 'BXP',
 'BSX',
 'BHF',
 'BMY',
 'AVGO',
 'BR',
 'BF.B',
 'CHRW',
 'CA',
 'COG',
 'CDNS',
 'CPB',
 'COF',
 'CAH',
 'KMX',
 'CCL',
 'CAT',
 'CBOE',
 'CBRE',
 'CBS',
 'CELG',
 'CNC',
 'CNP',
 'CTL',
 'CERN',
 'CF',
 'SCHW',
 'CHTR',
 'CVX',
 'CMG',
 'CB',
 'CHD',
 'CI',
 'XEC',
 'CINF',
 'CTAS',
 'CSCO',
 'C',
 'CFG',
 'CTXS',
 'CLX',
 'CME

### Getting all the data from the individual companies (!)

In [54]:
def data_from_yahoo(reload_sp500=True):
    if reload_sp500:
        tickers = save_sp500_tickers()
    else:
        with open("sp500tickers.pickle","rb") as f:
            tickers = pickle.load(f)
    
    if not os.path.exists('stock_dfs'):
        os.makedirs('stock_dfs')    
        
    start = dt.datetime(2000,1,1)
    finish = dt.datetime(2018, 9, 17)
    
    for ticker in tickers:
        if ticker.find('.'):
            ticker = ticker.replace('.','-')
        print(ticker)
        if not os.path.exists('stock_dfs/{}.csv'.format(ticker)):
            df = web.DataReader(ticker, 'yahoo', start, finish)
            df.to_csv('stock_dfs/{}.csv'.format(ticker))
        else:
            print('Already have {}'.format(ticker))
            

In [55]:
data_from_yahoo()

MMM
Already have MMM
ABT
Already have ABT
ABBV
Already have ABBV
ABMD
Already have ABMD
ACN
Already have ACN
ATVI
Already have ATVI
ADBE
Already have ADBE
AMD
Already have AMD
AAP
Already have AAP
AES
Already have AES
AET
Already have AET
AMG
Already have AMG
AFL
Already have AFL
A
Already have A
APD
Already have APD
AKAM
Already have AKAM
ALK
Already have ALK
ALB
Already have ALB
ARE
Already have ARE
ALXN
Already have ALXN
ALGN
Already have ALGN
ALLE
Already have ALLE
AGN
Already have AGN
ADS
Already have ADS
LNT
Already have LNT
ALL
Already have ALL
GOOGL
Already have GOOGL
GOOG
Already have GOOG
MO
Already have MO
AMZN
Already have AMZN
AEE
Already have AEE
AAL
Already have AAL
AEP
Already have AEP
AXP
Already have AXP
AIG
Already have AIG
AMT
Already have AMT
AWK
Already have AWK
AMP
Already have AMP
ABC
Already have ABC
AME
Already have AME
AMGN
Already have AMGN
APH
Already have APH
APC
Already have APC
ADI
Already have ADI
ANDV
Already have ANDV
ANSS
Already have ANSS
ANTM
Alrea

In [74]:
def compile_data():
    with open("sp500tickers.pickle", "rb") as f:
        tickers = pickle.load(f)

    main_df = pd.DataFrame()

    for count,ticker in enumerate(tickers):
        
        if ticker.find('.'):
            ticker = ticker.replace('.','-')
            
        df = pd.read_csv('stock_dfs/{}.csv'.format(ticker))
        df.set_index('Date', inplace=True)

        df.rename(columns={'Adj Close': ticker}, inplace=True)
        df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], 1, inplace=True)

        if main_df.empty:
            main_df = df
        else:
            main_df = main_df.join(df, how='outer')

        if count % 10 == 0:
            print(count)
            
    print(main_df.head())
    main_df.to_csv('sp500_joined_closes.csv')


### So, this below should be the way to continue this code, but it always crashes because of memory error. Will try o fix later.

In [75]:
compile_data()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500


In [76]:
main_df = pd.read_csv('sp500_joined_closes.csv')

In [77]:
main_df

,Date,MMM,ABT,ABBV,ABMD,ACN,ATVI,ADBE,AMD,AAP,...,WLTW,WYNN,XEL,XRX,XLNX,XYL,YUM,ZBH,ZION,ZTS
0,2000-01-03,27.668850,6.757675,NaN,18.250000,NaN,1.245554,16.145748,15.500000,NaN,...,NaN,NaN,8.113840,47.123608,34.193836,NaN,4.514584,NaN,42.999950,NaN
1,2000-01-04,26.569431,6.564601,NaN,17.812500,NaN,1.207666,14.791295,14.625000,NaN,...,NaN,NaN,8.300672,44.931831,33.416698,NaN,4.423840,NaN,40.917744,NaN
2,2000-01-05,27.339037,6.552534,NaN,18.000000,NaN,1.212402,15.083735,15.000000,NaN,...,NaN,NaN,8.620955,47.245369,32.685280,NaN,4.446526,NaN,40.869316,NaN
3,2000-01-06,29.537876,6.781809,NaN,18.031250,NaN,1.188722,15.206868,16.000000,NaN,...,NaN,NaN,8.540886,46.271240,29.851040,NaN,4.408716,NaN,41.450397,NaN
4,2000-01-07,30.124239,6.854216,NaN,17.937500,NaN,1.221874,15.945663,16.250000,NaN,...,NaN,NaN,8.540886,47.367149,33.233829,NaN,4.310407,NaN,41.547253,NaN
5,2000-01-10,29.977655,6.805944,NaN,20.500000,NaN,1.283441,16.561329,17.500000,NaN,...,NaN,NaN,8.540886,46.271240,34.925243,NaN,4.484337,NaN,40.796638,NaN
6,2000-01-11,29.464573,6.709409,NaN,19.812500,NaN,1.340273,15.422350,17.250000,NaN,...,NaN,NaN,8.540886,45.784180,32.136700,NaN,4.454087,NaN,38.641861,NaN
7,2000-01-12,29.537876,6.649129,NaN,19.593750,NaN,1.283441,15.345394,18.125000,NaN,...,NaN,NaN,8.727714,45.662415,32.136700,NaN,4.408716,NaN,38.980812,NaN
8,2000-01-13,29.537876,6.588129,NaN,21.000000,NaN,1.297649,16.161148,18.875000,NaN,...,NaN,NaN,8.701029,46.880077,33.096718,NaN,4.476774,NaN,42.370457,NaN
9,2000-01-14,29.116436,6.734531,NaN,21.375000,NaN,1.283441,16.330452,20.187500,NaN,...,NaN,NaN,8.727714,45.905952,35.245251,NaN,4.355781,NaN,43.096802,NaN
